In [ ]:
!pip install docx2txt
!pip install tika

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=53e275723499e8ca076a5c2dc50ba489ac97e5445439b146d09dd3b7ac7c59a1
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=bac77f96791763db94fd834dce7e28464a654624e62e8fff9ac7dd60c25aaaf7
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [ ]:
!pip install pyngrok
!pip install Flask_wtf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 4.2 MB/s eta 0:00:00


In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wtforms

In [ ]:
port_no = 5000

In [10]:
from flask import Flask, render_template, request
from flask_wtf import FlaskForm
from wtforms import FileField, SubmitField
from wtforms.validators import InputRequired
from werkzeug.utils import secure_filename
import os
import re
import nltk
import spacy
import docx2txt
from tika import parser

nltk.download("punkt")
nltk.download("stopwords")

def extract_text_from_pdf(file_path):
    try:
        parsed_pdf = parser.from_file(file_path)
        return parsed_pdf['content']
    except Exception as e:
        print(f"Error extracting text from {file_path}: {str(e)}")
        return ""

#function for extracting text from documents
def extract_text_from_doc(file_path):
    try:
        text = docx2txt.process(file_path)
        return text
    except Exception as e:
        print(f"Error extracting text from {file_path}: {str(e)}")
        return ""

def preprocess(txt):
    txt = txt.lower() if isinstance(txt, str) else ' '.join(txt).lower() #make lower
    txt = re.sub(r'\n', ' ', txt) # remove /n
    txt = re.sub('(http\S+|www.\S+)', ' ', txt)  # Remove URLs
    txt = re.sub('#\S+', '', txt) # remove #
    txt = re.sub(r'[\/,.:❖•;]', ' ', txt)  # Remove the specified special characters
    txt = re.sub('\s+', ' ', txt) # replace multiple whitespace characters (such as spaces, tabs, newlines, etc.)
    txt = re.sub(r'', '', txt) # remove 
    txt = nltk.tokenize.word_tokenize(txt) # tokenize the text
    txt = [w for w in txt if not w in nltk.corpus.stopwords.words('english')]

    return ' '.join(txt)

from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
# Function to extract name from resume text
matcher = Matcher(nlp.vocab)

def extract_name(text):
    nlp_text = nlp(text)
    # Define pattern for first and last name as Proper Nouns
    pattern = [[{'POS': 'PROPN'}, {'POS': 'PROPN'}]]
    matcher.add('NAME', pattern)
    matches = matcher(nlp_text)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

# functions to extract mob number
def extract_phone_numbers(text):
    phone_pattern = r" (?:\d[ -]?){9,15} "
    return re.findall(phone_pattern, text)

# Function to extract email id
def extract_email_addresses(text):
    email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
    match = email_pattern.search(text)
    return match.group() if match else None

#function for extracting skills
def extract_skills(text):
    skills_keywords = ['skills', 'technical skills', 'personal skills']
    end_keywords = ['contact', 'profile', 'interest', 'internship', 'education', 'work history', 'work', 'experience', 'summary', 'achievement', 'achievements', 'certifications', 'projects', 'accomplishments', 'accomplishment', 'work experience', 'qualification']  # Keywords to stop extraction

    skill_sections = []
    used_keywords = set()  # Store used keywords to avoid duplicates
    last_end_index = 0  # Store the last end index to begin searching from there

# finds the keywords and extract the content in that
    for keyword in skills_keywords:
        if keyword not in used_keywords:
            start_index = text.find(keyword, last_end_index)
            if start_index != -1:
                end_index = len(text)
                for end_key in end_keywords:
                    end_index_candidate = text.find(end_key, start_index + len(keyword))
                    if end_index_candidate != -1 and end_index_candidate < end_index:
                        end_index = end_index_candidate
                extracted_text = text[start_index + len(keyword):end_index].strip()
                if extracted_text:
                    skill_sections.append(extracted_text)
                used_keywords.add(keyword)
                last_end_index = end_index

    return skill_sections

#function for extracting education
def extract_education(text):
    education_keywords = ['education', 'degree', 'university', 'college', 'qualification']
    end_keywords = ['contact', 'profile', 'interest', 'internship', 'languages', 'skills', 'work history', 'experience', 'summary', 'achievement', 'achievements', 'certifications', 'projects', 'accomplishments', 'accomplishment', 'work experience']  # Keywords to stop extraction

    education_sections = []
    used_keywords = set()
    last_end_index = 0

# finds the keywords and extract the content in that
    for keyword in education_keywords:
        if keyword not in used_keywords:
            start_index = text.find(keyword, last_end_index)
            if start_index != -1:
                end_index = len(text)
                for end_key in end_keywords:
                    end_index_candidate = text.find(end_key, start_index + len(keyword))
                    if end_index_candidate != -1 and end_index_candidate < end_index:
                        end_index = end_index_candidate
                extracted_text = text[start_index + len(keyword):end_index].strip()
                if extracted_text:
                    education_sections.append(extracted_text)
                used_keywords.add(keyword)
                last_end_index = end_index

    return education_sections

#function for extracting work experience
def extract_work_experience(text):
    experience_keywords = ['internship', 'professional experience', 'experience', 'work experience', 'employment', 'work history']
    end_keywords = ['contact', 'profile', 'interest', 'skills', 'languages', 'technical skills', 'education', 'summary', 'achievement', 'certifications', 'projects', 'accomplishments', 'accomplishment', 'qualification']  # Keywords to stop extraction

    experience_sections = []
    used_keywords = set()
    last_end_index = 0

#finds the keywords and extract content in that
    for keyword in experience_keywords:
        if keyword not in used_keywords:
            start_index = text.find(keyword, last_end_index)
            if start_index != -1:
                end_index = len(text)
                for end_key in end_keywords:
                    end_index_candidate = text.find(end_key, start_index + len(keyword))
                    if end_index_candidate != -1 and end_index_candidate < end_index:
                        end_index = end_index_candidate

                extracted_text = text[start_index + len(keyword):end_index].strip()  # Exclude the keyword itself
                if extracted_text:  # Check if the extracted text is not empty
                    experience_sections.append(extracted_text)
                used_keywords.add(keyword)
                last_end_index = end_index

    return experience_sections

def extract_details(file_path):
    text = ""
    if file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith('.docx'):
        text = extract_text_from_doc(file_path)

    processed_text = preprocess(text)

    # Replace these lines with the function calls you want to use
    name = extract_name(processed_text)
    phone_numbers = extract_phone_numbers(processed_text)
    email_address = extract_email_addresses(text)
    skills = extract_skills(processed_text)
    education = extract_education(processed_text)
    work_experience = extract_work_experience(processed_text)

    extracted_data = {
        'name': name,
        'phone_numbers': phone_numbers,
        'email_address': email_address,
        'skills': skills,
        'education': education,
        'work_experience': work_experience
    }
    return extracted_data

def format_extracted_data(extracted_data):

    formatted_data = {
        'name': extracted_data['name'],
        'phone_numbers': ', '.join([number.strip() for number in extracted_data['phone_numbers']]) if extracted_data['phone_numbers'] else None,
        'email_address': extracted_data['email_address'],
        'skills': ', '.join(extracted_data['skills']) if extracted_data['skills'] else None,
        'education': ', '.join(extracted_data['education']) if extracted_data['education'] else None,
        'work_experience': ', '.join(extracted_data['work_experience']) if extracted_data['work_experience'] else None
    }
    return formatted_data

# Flask app setup
template_folder = "/content/drive/MyDrive/Resume_Parser/template"
app = Flask(__name__, template_folder=template_folder)
ngrok.set_auth_token("2gN7hnjcZeiu6vJCaLj8ZTj4oBz_58BGx3VyXXwSXYi54vo7k")
public_url =  ngrok.connect(port_no).public_url
app.config['SECRET_KEY'] = 'supersecretkey'
app.config['UPLOAD_FOLDER'] = '/content/drive/MyDrive/Resume_Parser/static/file'

# Define your FlaskForm for file upload
class UploadFileForm(FlaskForm):
    file = FileField("File", validators=[InputRequired()])
    submit = SubmitField("Upload File")

# Route for uploading resumes and processing them
@app.route('/', methods=['GET', 'POST'])
@app.route('/home', methods=['GET', 'POST'])
def home():
    form = UploadFileForm()
    extracted_data = None

    if form.validate_on_submit():
        file = form.file.data
        if file:
            file_path = "uploads/" + file.filename
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename)))
            extracted_data = extract_details(os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename)))
            extracted_data = format_extracted_data(extracted_data)  # Format extracted data here

            return render_template('index.html', form=form, extracted_data=extracted_data)

    return render_template('index.html', form=form, extracted_data=extracted_data)

print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


To acces the Gloable link please click https://6df6-34-125-245-119.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/May/2024 16:11:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2024 16:11:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/May/2024 16:11:24] "POST / HTTP/1.1" 200 -
